## RNN 활용 - 단어 자동 완성하기

#### 4개의 글자를 가진 단어를 학습시켜, 3글자만 주어지면 나머지 한글자를 추천하여 단어를 완성
 * dynamic_rnn의 sequence_length 옵션을 사용하면 가변 길이의 단어를 학습
 * 학습시킬 데이터는 영문자로 구성된 임의의 단어 사용
 * 한 글자, 한 글자가 한 단계의 입력값, 총 글자수가 전체 단계가 된다.
 * word : 4글자 w, o, r, d - 총 4단계

In [1]:
import tensorflow as tf
import numpy as np

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import os, warnings
# 이미지 처리 분야에서 가장 유명한 신경망 모델인 CNN 을 이용
import tensorflow as tf
import time
from IPython.display import display, Image

In [3]:
# 경고 메시지 무시하거나 숨길때(ignore), 다시보이게(default)
# warnings.filterwarnings(action='default')
warnings.filterwarnings(action='ignore')

### 알파벳 리스트 지정

In [4]:
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

### 알파벳을 숫자로 보여주기

* one-hot 인코딩 사용 및 디코딩을 하기 위해 연관 배열을 생성.
* 알파벳 딕셔너리 데이터 셋 생성.

In [5]:
# one-hot 인코딩 사용 및 디코딩을 하기 위해 연관 배열을 만듭니다.
# {'a': 0, 'b': 1, 'c': 2, ..., 'j': 9, 'k', 10, ...}
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)
print(dic_len)

26


### 테스트를 위해 데이터 준비

In [6]:
# 입력값과 출력값으로 아래와 같이 사용
# wor -> X, d -> Y
# woo -> X, d -> Y
seq_data = ['word', 'wood', 'deep', 'dive', 
            'cold', 'cool', 'load', 'love', 'kiss', 'kind']

In [7]:
# 각 단어의 한글자 한글자 index를 저장
print([num_dic[n] for n in seq_data[0][:-1]])

print(np.eye(dic_len)[[22, 14, 17]])

[22, 14, 17]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0.]]


### def make_batch(seq_data) : 주어진 데이터를 입력과 출력으로 분리
* input_batch = [], target_batch = [] 입출력 리스트 
* for seq in seq_data: -> 단어별 seq 전달
    
* 각 단어의 한글자 한글자 index를 저장(4글자중 3글자)
   * input = [num_dic[n] for n in seq[:-1]] => i
    
* 단어의 마지막 글자 인덱스
   * target = num_dic[seq[-1]]

* 3글자의 원핫인코딩값을 추가
   * input_batch.append(np.eye(dic_len)[input])

* 4번째 글자는 원핫하지 않고, 넘겨준다.
   * target_batch.append(target)

In [8]:
def make_batch(seq_data, dic_len_num):
    input_batch = []
    target_batch = []

    for seq in seq_data:
        # input은 단어의 글자 알파벳 배열의 인덱스 번호 입니다.
        # [22, 14, 17] [22, 14, 14] [3, 4, 4] [3, 8, 21] ...
        input = [num_dic[n] for n in seq[:-1]]
        
        target = num_dic[seq[-1]]  # 마지막 글자 인덱스
        
        # 3글자에 대한 one-hot 인코딩을 합니다.
        # if input is [0, 1, 2]:
        # [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        #  [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
        #  [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]]
        input_batch.append(np.eye(dic_len_num)[input])
        target_batch.append(target)

    return input_batch, target_batch

* 지금까지 손실함수로 사용하던 softmax_cross_entropy_with_logits 함수는
* label 값을 one-hot 인코딩으로 넘겨줘야 하지만,
* 이 예제에서 사용할 손실 함수인 
* sparse_softmax_cross_entropy_with_logits 는
* one-hot 인코딩을 사용하지 않으므로 index 를 그냥 넘겨주면 됩니다.

In [9]:
seq_data_test = ['work', 'book']

In [10]:
## 잠시 결과 확인
## word
a, b= make_batch(seq_data_test, dic_len)
print("a의 값 : 3글자의 알파벳 인덱스 원핫값")  # w, o, r
print("전체 단어수 : {}".format(len(a)))
print("첫번째 단어 3글자 :")
print(a[0])  # w, o, r
print("두번째 단어 3글자 :")
print(a[1])  # b, o, o
print("b의 값 : 마지막 글자 index")  # k, k
print(b)  # d
print("a와 b의 리스트 길이")  # k, k
print(len(a), len(b))  # d

a의 값 : 3글자의 알파벳 인덱스 원핫값
전체 단어수 : 2
첫번째 단어 3글자 :
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0.]]
두번째 단어 3글자 :
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]
b의 값 : 마지막 글자 index
[10, 10]
a와 b의 리스트 길이
2 2


### 파라미터 설정
* learning_rate = 0.01 (학습률)
* n_hidden = 128       (은닉층 노드 개수)
* total_epoch = 30     (전체 학습 epoch 수)
* n_step = 3           (RNN구성 Step 수)
* 입력값 크기, 알파벳 one-hot 인코딩 개수
  * n_input = n_class = dic_len
  * 예) c => [0 0 1 0 0 0 0 0 0 0 0 ... 0]
  * 출력값도 입력값과 마찬가지로 26개의 알파벳으로 분류.

In [11]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 30

n_step = 3
n_input = n_class = dic_len

### 신경망 모델 구성
* Y의 값을 정수 선언
  * 비용함수에 sparse_softmax_cross_entropy_with_logits 을 사용
  * one-hot vector 의 형태가 아닌, 인덱스 숫자를 그대로 사용

### sparse_softmax_cross_entropy_with_logits 

<pre>
tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels,
    logits,
    name=None
)
</pre>
*  labels 값은 인덱스 값 숫자를 그대로 사용
* 예측 모델의 출력값은 인덱스의 원핫 인코딩을 사용

In [12]:
X = tf.placeholder(tf.float32, [None, n_step, n_input]) # 입력 
Y = tf.placeholder(tf.int32, [None])                    # 출력
W = tf.Variable(tf.random_normal([n_hidden, n_class]))  
b = tf.Variable(tf.random_normal([n_class]))

Instructions for updating:
Colocations handled automatically by placer.


### RNN 셀을 생성

* DropoutWrapper 함수 : RNN에도 과적합 방지를 위한 DropoutWrapper 함수 적용이 가능하다. 
* 두개의 RNN 셀 생성 : 여러셀을 조합한 심층 신경망 만들기
  * tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
  * DropOut 사용 
    * tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)

In [13]:
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden) # 첫번째 셀 생성
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)  # Dropout

cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden) # 두번째 셀 생성

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


### Deep RNN을 만들기 
* 앞에서 만든 셀을 조합한다. (MultiRNNCell 함수 사용)
* dynamic_rnn 함수 사용
  * dynamic_rnn 함수 : Creates a recurrent neural network specified by RNNCell cell
  * RNN셀을 활용하여 RNN 신경망을 만든다.
  * https://www.tensorflow.org/versions/r1.14/api_docs/python/tf/nn/dynamic_rnn

In [14]:
# 에러 발생시 커널 restart
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2]) # 멀티 셀 생성.

outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32) # RNN신경망

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [15]:
print("multi_cell ======>")
print(multi_cell)
print()
print("outputs ======>")
print(outputs)

print()
print("states ======>")
print(states)

multi_cell ======>

outputs ======>
Tensor("rnn/transpose_1:0", shape=(?, 3, 128), dtype=float32)

states ======>
(LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 128) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 128) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_5:0' shape=(?, 128) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_6:0' shape=(?, 128) dtype=float32>))


### 최종 출력층 만들기
* RNN 신경망이 출력값의 형태 -> [batch_size, n_step, n_hidden]
* 이를 실측값과 비교를 위해 아래와 같이 변경
  * (가) 순서 바꾸기 :  
  * [batch_size, n_step, n_hidden]  -> [n_step, batch_size, n_hidden]
  * (나) n_step의 차원을 제거 [batch_size, n_hidden]

In [16]:
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

### 손실함수(cost), 최적화 함수(optimizer)

In [17]:
cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### 신경망 실행 - 그래프 실행

In [18]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data, dic_len)

In [19]:
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={X: input_batch, Y: target_batch})

    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))

print('최적화 완료!')

Epoch: 0001 cost = 4.248590
Epoch: 0002 cost = 3.131035
Epoch: 0003 cost = 1.843633
Epoch: 0004 cost = 1.450798
Epoch: 0005 cost = 0.990852
Epoch: 0006 cost = 0.810740
Epoch: 0007 cost = 0.384700
Epoch: 0008 cost = 0.468316
Epoch: 0009 cost = 0.306273
Epoch: 0010 cost = 0.217975
Epoch: 0011 cost = 0.283368
Epoch: 0012 cost = 0.205272
Epoch: 0013 cost = 0.209421
Epoch: 0014 cost = 0.163986
Epoch: 0015 cost = 0.181547
Epoch: 0016 cost = 0.180372
Epoch: 0017 cost = 0.102199
Epoch: 0018 cost = 0.034223
Epoch: 0019 cost = 0.068512
Epoch: 0020 cost = 0.048412
Epoch: 0021 cost = 0.104109
Epoch: 0022 cost = 0.048344
Epoch: 0023 cost = 0.050188
Epoch: 0024 cost = 0.066766
Epoch: 0025 cost = 0.166204
Epoch: 0026 cost = 0.004133
Epoch: 0027 cost = 0.036584
Epoch: 0028 cost = 0.005542
Epoch: 0029 cost = 0.111795
Epoch: 0030 cost = 0.200482
최적화 완료!


### 결과 확인

In [20]:
# 레이블값이 정수이므로 예측값도 정수로 변경해줍니다.
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
print("prediction : ", prediction)
# one-hot 인코딩이 아니므로 입력값을 그대로 비교합니다.
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))
print("accuracy : ", accuracy)

prediction :  Tensor("Cast:0", shape=(?,), dtype=int32)
accuracy :  Tensor("Mean_1:0", shape=(), dtype=float32)


In [21]:
seq_data_test = ['hard', 'okay', 'deep', 'live', 
            'call', 'cool', 'load', 'love', 'kiss', 'kind']

In [22]:
input_batch, target_batch = make_batch(seq_data_test, dic_len)

predict, accuracy_val = sess.run([prediction, accuracy],
                                 feed_dict={X: input_batch, Y: target_batch})

print(predict, accuracy_val)

[ 3  3 15  4  3 11  3  4 18  3] 0.8


In [23]:
predict_words = []
for idx, val in enumerate(seq_data_test):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)

print('\n=== 예측 결과 ===')
print('입력값:', [w[:3] + ' ' for w in seq_data_test])
print('예측값:', predict_words)
print('정확도:', accuracy_val)


=== 예측 결과 ===
입력값: ['har ', 'oka ', 'dee ', 'liv ', 'cal ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['hard', 'okad', 'deep', 'live', 'cald', 'cool', 'load', 'love', 'kiss', 'kind']
정확도: 0.8


### 실습해 보기
* seq_data 의 학습 사전을 100단어 정도로 더 업그레이드를 해 보자.
* 그리고 실제 웹을 검색 후, 텍스트에서 4개 단어가 되는 것을 가지고 예측을 수행해보자.
* 정확도가 어느정도 되는가?

### REF
* sparse_softmax_cross_entropy_with_logits
https://www.tensorflow.org/api_docs/python/tf/nn/sparse_softmax_cross_entropy_with_logits